In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime

In [ ]:
yf.pdr_override()
start_date = datetime(2022, 6, 1)
end_date = datetime(2024, 6, 1)
acoes = ["PETR4.SA","VALE3.SA"]
df_acoes = []
for acao in acoes:
    df = pdr.get_data_yahoo(acao, start=start_date, end=end_date)
    df.index = pd.to_datetime(df.index) # indice nao vem como datetime
    df['stock'] = acao[:-3]
    df_acoes.append(df)

df_prices = pd.concat(df_acoes)

In [6]:
df_prices

,Open,High,Low,Close,Adj Close,Volume,stock
Date,,,,,,,
2022-06-01,29.900000,30.330000,29.850000,30.020000,16.022127,46353200,PETR4
2022-06-02,29.969999,30.049999,29.549999,29.760000,15.883363,50738300,PETR4
2022-06-03,29.709999,30.440001,29.530001,30.280001,16.160894,60396900,PETR4
2022-06-06,30.500000,30.530001,29.910000,30.299999,16.171568,39213900,PETR4
2022-06-07,30.190001,31.299999,30.160000,30.660000,16.363707,80333700,PETR4
...,...,...,...,...,...,...,...
2024-05-24,65.000000,65.660004,64.900002,65.080002,65.080002,11731900,VALE3
2024-05-27,64.910004,65.300003,64.709999,65.300003,65.300003,5119400,VALE3
2024-05-28,65.279999,65.290001,63.349998,63.889999,63.889999,22173900,VALE3


In [2]:
PATH_NLI_FEATURES_AGG_BY_DAY = 'data/additonal_features/tweets_keywords_relations_agg_by_day.csv'

In [3]:
df_nli_features = pd.read_csv(PATH_NLI_FEATURES_AGG_BY_DAY)

In [4]:
df_nli_features

,stock,valorização_mean,valorização_median,valorização_q1,valorização_q3,valorização_count,bom negócio_mean,bom negócio_median,bom negócio_q1,bom negócio_q3,...,prejuízo_q1,prejuízo_q3,prejuízo_count,governo_mean,governo_median,governo_q1,governo_q3,governo_count,has_tweets,datetime
0,PETR4,0.501828,0.501828,0.253213,0.750443,2,0.491784,0.491784,0.367215,0.616352,...,0.077469,0.121519,2,0.887368,0.887368,0.886389,0.888347,2,0,2022-06-01
1,PETR4,0.501828,0.501828,0.253213,0.750443,2,0.491784,0.491784,0.367215,0.616352,...,0.077469,0.121519,2,0.887368,0.887368,0.886389,0.888347,2,0,2022-06-02
2,PETR4,0.501828,0.501828,0.253213,0.750443,2,0.491784,0.491784,0.367215,0.616352,...,0.077469,0.121519,2,0.887368,0.887368,0.886389,0.888347,2,0,2022-06-03
3,PETR4,0.501828,0.501828,0.253213,0.750443,2,0.491784,0.491784,0.367215,0.616352,...,0.077469,0.121519,2,0.887368,0.887368,0.886389,0.888347,2,0,2022-06-04
4,PETR4,0.501828,0.501828,0.253213,0.750443,2,0.491784,0.491784,0.367215,0.616352,...,0.077469,0.121519,2,0.887368,0.887368,0.886389,0.888347,2,0,2022-06-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1457,VALE3,0.588541,0.748972,0.411575,0.812859,5,0.553513,0.663868,0.639569,0.689314,...,0.013599,0.039807,5,0.456577,0.528233,0.402417,0.644755,5,1,2024-05-27
1458,VALE3,0.702186,0.724803,0.562714,0.864276,4,0.737152,0.702927,0.636298,0.803780,...,0.028030,0.841715,4,0.280694,0.079572,0.017608,0.342658,4,1,2024-05-28
1459,VALE3,0.532552,0.532929,0.071492,0.993988,4,0.641740,0.769118,0.572464,0.838395,...,0.252097,0.918396,4,0.681841,0.755330,0.474074,0.963098,4,1,2024-05-29
1460,VALE3,0.999218,0.999218,0.999218,0.999218,1,0.884908,0.884908,0.884908,0.884908,...,0.278165,0.278165,1,0.560456,0.560456,0.560456,0.560456,1,1,2024-05-30
